In [40]:
users = ["User1", "User2", "User3", "User4", "User5"]
items = ["Item A", "Item B", "Item C", "Item D", "Item E"]
datasets = [
    [1, 0, 1, 1, 0],
    [1, 0, 1, 1, 1],
    [0, 1, 0, 0, 0],
    [0, 1, 1, 0, 1],
    [1, 1, 0, 1, 1],
]

import pandas as pd
df = pd.DataFrame(datasets, columns=items, index=users)
print(df)

       Item A  Item B  Item C  Item D  Item E
User1       1       0       1       1       0
User2       1       0       1       1       1
User3       0       1       0       0       0
User4       0       1       1       0       1
User5       1       1       0       1       1


In [41]:
import pandas as pd
from sklearn.metrics import jaccard_score
from sklearn.metrics.pairwise import pairwise_distances

# 将数据框转换为布尔型
df = df.astype(bool)

# 计算 Item A 和 Item B 的 Jaccard 相似度
jaccard_score_ab = jaccard_score(df["Item A"], df["Item B"])
print(f"Item A 和 Item B 的 Jaccard 相似度: {jaccard_score_ab}")

Item A 和 Item B 的 Jaccard 相似度: 0.2


In [42]:
df.values

array([[ True, False,  True,  True, False],
       [ True, False,  True,  True,  True],
       [False,  True, False, False, False],
       [False,  True,  True, False,  True],
       [ True,  True, False,  True,  True]])

In [43]:
# 计算用户之间的 Jaccard 相似度
user_similar = 1 - pairwise_distances(df.values, metric="jaccard")
user_similar_df = pd.DataFrame(user_similar, columns=users, index=users)
print("用户之间的 Jaccard 相似度：")
print(user_similar_df)

# 计算物品之间的 Jaccard 相似度
item_similar = 1 - pairwise_distances(df.T.values, metric="jaccard")
item_similar_df = pd.DataFrame(item_similar, columns=items, index=items)
print("物品之间的 Jaccard 相似度：")
print(item_similar_df)

用户之间的 Jaccard 相似度：
       User1  User2     User3     User4  User5
User1   1.00   0.75  0.000000  0.200000   0.40
User2   0.75   1.00  0.000000  0.400000   0.60
User3   0.00   0.00  1.000000  0.333333   0.25
User4   0.20   0.40  0.333333  1.000000   0.40
User5   0.40   0.60  0.250000  0.400000   1.00
物品之间的 Jaccard 相似度：
        Item A  Item B  Item C  Item D  Item E
Item A     1.0     0.2     0.5     1.0     0.5
Item B     0.2     1.0     0.2     0.2     0.5
Item C     0.5     0.2     1.0     0.5     0.5
Item D     1.0     0.2     0.5     1.0     0.5
Item E     0.5     0.5     0.5     0.5     1.0


In [44]:
topN_users = {}

# 遍历每一行数据
for user in user_similar_df.index:
    # 获取相似度，删除自己的相似度，然后按相似度排序
    _df = user_similar_df.loc[user].drop(user)
    _df_sorted = _df.sort_values(ascending=False)
    # 获取前两个相似用户
    top2 = list(_df_sorted.index[:2])
    topN_users[user] = top2


In [45]:
print("Top2相似用户：")
print(topN_users)

Top2相似用户：
{'User1': ['User2', 'User5'], 'User2': ['User1', 'User5'], 'User3': ['User4', 'User5'], 'User4': ['User2', 'User5'], 'User5': ['User2', 'User1']}


In [57]:
rs_results = {}

# 遍历每个用户及其相似用户
for user, sim_users in topN_users.items():
    rs_result = set()  # 为每个用户保存推荐结果
    
    for sim_user in sim_users:
        # 获取相似用户的评分
        bought_items = df.loc[sim_user]
        print(f"{sim_user}的评分: {bought_items.to_list()}")
        
        # 进行初步推荐（提取所有已购买的物品）
        rs_result = rs_result.union(set(bought_items[bought_items == 1].index))
    
    # 过滤掉用户已经购买过的物品
    purchased_items = set(df.loc[user][df.loc[user] == 1].index)
    rs_result -= purchased_items
    
    print(f"过滤后推荐结果: {rs_result}")
    
    rs_results[user] = rs_result  # 存储最终推荐结果

User2的评分: [True, False, True, True, True]
User5的评分: [True, True, False, True, True]
过滤后推荐结果: {'Item E', 'Item B'}
User1的评分: [True, False, True, True, False]
User5的评分: [True, True, False, True, True]
过滤后推荐结果: {'Item B'}
User4的评分: [False, True, True, False, True]
User5的评分: [True, True, False, True, True]
过滤后推荐结果: {'Item E', 'Item D', 'Item A', 'Item C'}
User2的评分: [True, False, True, True, True]
User5的评分: [True, True, False, True, True]
过滤后推荐结果: {'Item D', 'Item A'}
User2的评分: [True, False, True, True, True]
User1的评分: [True, False, True, True, False]
过滤后推荐结果: {'Item C'}


In [58]:
rs_results

{'User1': {'Item B', 'Item E'},
 'User2': {'Item B'},
 'User3': {'Item A', 'Item C', 'Item D', 'Item E'},
 'User4': {'Item A', 'Item D'},
 'User5': {'Item C'}}